In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf

In [2]:
# Code Needs: (eventually)
# - ROICaT index alignment tools (will wait until I have data processed and loaded from multiple sessions)

In [3]:
# Immediate next steps for programming work:

# - red cell processing functions
# - database development


In [12]:
vrFileName = vrReg.sessionPath() / f"{vrReg.dateString}_{vrReg.session}_{vrReg.mouseName}_VRBehavior_trial.mat" # vrBehavior output file name
# vrFile = sp.io.loadmat(vrFileName,simplify_cells=True) #,simplify_cells=True) # load matlab structure
vrFile = sp.io.loadmat(vrFileName,struct_as_record=False,squeeze_me=True)


In [14]:
vrFile['expInfo']

In [11]:
# vrExperiment registration and preprocessing 
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL022/2023-03-24/701, failed to load vrFile as a simplified_cell due to the 'getEnvName' anonymous function. Continuing with struct_as_record==False and squeeze_me==True
The problem can be solved by adding a boolean statement to _has_struct(elem) in scipy.io.matlab._mio5.py, make sure (elem.ndim>0) before asking for elem[0]
In session: ATL022/2023-03-24/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!


TypeError: 'mat_struct' object is not subscriptable

In [56]:
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [60]:
%%time
omap, smap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=5) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

Lick map should be included in 'getBehaviorMaps' eventually!
CPU times: total: 5.78 s
Wall time: 506 ms
